<a href="https://colab.research.google.com/github/konpatel/Pneumonia-detection-xray/blob/master/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout,GlobalAveragePooling2D
from keras.models import Model,load_model,Sequential
# import efficientnet.keras as efn
import keras.backend as K

from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.applications import VGG19


In [3]:
traindf=pd.read_csv("/content/drive/MyDrive/MachineLearningImages/labels_train.csv",dtype=str)
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.25,
        horizontal_flip=True,
        validation_split=0.2,
        fill_mode='nearest')

In [4]:
#Διάσταση κάθε εικόνας
IMAGE_SIZE    = (224, 224)
#Αριθμός κατηγοριών που θα γίνει το classification
NUM_CLASSES   = 3
#Σύνολο εικόνων που θα τροφοδοτούμε το μοντέλο κάθε φορά
BATCH_SIZE    = 16  
#Αριθμός layer που θα αποκλείσουμε από το CNN
FREEZE_LAYERS = 2  
#Αριθμός Εποχών
NUM_EPOCHS    = 10
#Filename του μοντέλου
WEIGHTS_FINAL = 'model-final.h5'

In [5]:
train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/MyDrive/MachineLearningImages/train_images/",
    x_col="file_name",
    y_col="class_id",
    subset="training",
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=IMAGE_SIZE)

valid_generator =datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/MyDrive/MachineLearningImages/train_images",
    x_col="file_name",
    y_col="class_id",
    subset="validation",
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=IMAGE_SIZE)

Found 3738 validated image filenames belonging to 3 classes.
Found 934 validated image filenames belonging to 3 classes.


In [6]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_directory(    
    directory="/content/drive/MyDrive/MachineLearningImages/test_images/",    
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=IMAGE_SIZE)

# fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# images = test_generator.next()
# for i in range(4):
#     for j in range(4):
#         image_new = images[i+j]
#         ax[i][j].imshow(image_new, cmap='gray')
#         ax[i][j].axis('off')

# plt.show()

Found 1168 images belonging to 1 classes.


In [7]:
from tensorflow.keras.layers import Input
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation

In [8]:
# conv_model = ResNet50(weights=None, include_top=False, input_shape=(224,224,3))
# for layer in conv_model.layers:
#     layer.trainable = False
# x = Flatten()(conv_model.output)
# x = Dense(100, activation='relu')(x)
# x = Dense(100, activation='relu')(x)
# x = Dense(100, activation='relu')(x)
# predictions = Dense(3, activation='softmax')(x)
# full_model = Model(inputs=conv_model.input, outputs=predictions)
# full_model.compile(optimizer = Adam(learning_rate=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
# full_model.summary()



inputs = Input(shape=(224, 224, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=1, activation='sigmoid')(x)

# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [ ]:
max_epochs = 10
batch_size = 32


model.fit(train_generator,
                        validation_data = valid_generator,
                        batch_size=batch_size,
                        epochs=max_epochs,
                        verbose=1)

# save trained weights
model.save(WEIGHTS_FINAL)

Epoch 1/10
 78/234 [=========>....................] - ETA: 12:33 - loss: 0.6442 - accuracy: 0.6653